<a href="https://colab.research.google.com/github/cristianmunoz1/ai4eng-proyecto/blob/main/03%20-%20preprocesado-test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Replicamos los recursos locales en el entorno virtual

In [1]:
!wget --no-cache -O init.py -q https://raw.githubusercontent.com/rramosp/ai4eng.v1/main/content/init.py
import init; init.init(force_download=False); init.get_weblink()

replicating local resources


### Descargamos los datos de la competencia al entorno de trabajo

In [2]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai4eng-20242

chmod: cannot access './kaggle.json': No such file or directory
 70% 14.0M/20.1M [00:00<00:00, 145MB/s]
100% 20.1M/20.1M [00:00<00:00, 144MB/s]


### Descomprimimos el archivo

In [3]:
!unzip udea*.zip > /dev/null

In [ ]:
!wc *.csv

   296787    296787   4716673 submission_example.csv
   296787   4565553  50135751 test.csv
   692501  10666231 118025055 train.csv
  1286075  15528571 172877479 total


## Importe de librerías necesarias

In [4]:
!pip install fuzzywuzzy
!pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 5.0 MB/s eta 0:00:00


In [5]:
import unidecode as ud
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz, process
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


### Cargamos el csv como un dataframe de Pandas
Se guarda en la variable train

In [8]:
test = pd.read_csv("test.csv")
print ("shape of loaded dataframe", test.shape)

shape of loaded dataframe (296786, 12)


In [9]:
test.columns

Index(['Unnamed: 0', 'ID', 'PERIODO', 'ESTU_PRGM_ACADEMICO',
       'ESTU_PRGM_DEPARTAMENTO', 'ESTU_VALORMATRICULAUNIVERSIDAD',
       'ESTU_HORASSEMANATRABAJA', 'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET',
       'FAMI_EDUCACIONPADRE', 'FAMI_EDUCACIONMADRE',
       'ESTU_PAGOMATRICULAPROPIO'],
      dtype='object')

In [10]:
test.head()

,Unnamed: 0,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO
0,0,550236,20183,TRABAJO SOCIAL,BOLIVAR,Menos de 500 mil,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica completa,Primaria completa,Si
1,1,98545,20203,ADMINISTRACION COMERCIAL Y DE MERCADEO,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Técnica o tecnológica completa,No
2,2,499179,20212,INGENIERIA MECATRONICA,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,0,Estrato 3,Si,Secundaria (Bachillerato) incompleta,Secundaria (Bachillerato) completa,No
3,3,782980,20195,CONTADURIA PUBLICA,SUCRE,Entre 1 millón y menos de 2.5 millones,Entre 21 y 30 horas,Estrato 1,No,Primaria incompleta,Primaria incompleta,No
4,4,785185,20212,ADMINISTRACION DE EMPRESAS,ATLANTICO,Entre 2.5 millones y menos de 4 millones,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,No


## One_hot encoding para la columna 'ESTU_PRGMACADEMICO'

In [11]:
carreras_unicas = test['ESTU_PRGM_ACADEMICO'].unique()

In [12]:
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].apply(lambda x: ud.unidecode(x.lower()))

In [13]:
test.head()

,Unnamed: 0,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO
0,0,550236,20183,trabajo social,BOLIVAR,Menos de 500 mil,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica completa,Primaria completa,Si
1,1,98545,20203,administracion comercial y de mercadeo,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Técnica o tecnológica completa,No
2,2,499179,20212,ingenieria mecatronica,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,0,Estrato 3,Si,Secundaria (Bachillerato) incompleta,Secundaria (Bachillerato) completa,No
3,3,782980,20195,contaduria publica,SUCRE,Entre 1 millón y menos de 2.5 millones,Entre 21 y 30 horas,Estrato 1,No,Primaria incompleta,Primaria incompleta,No
4,4,785185,20212,administracion de empresas,ATLANTICO,Entre 2.5 millones y menos de 4 millones,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,No


In [14]:
counts = test['ESTU_PRGM_ACADEMICO'].value_counts().to_frame()
prgm_acads = (counts.loc[counts['count'] > 6000]).index
prgm_acads
len(prgm_acads)

9

In [15]:
array = [element for element in test['ESTU_PRGM_ACADEMICO'] if element not in prgm_acads ]
array

condition = test['ESTU_PRGM_ACADEMICO'].isin(array)
test.loc[condition, 'ESTU_PRGM_ACADEMICO'] = "otros"
test.head()

,Unnamed: 0,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO
0,0,550236,20183,otros,BOLIVAR,Menos de 500 mil,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica completa,Primaria completa,Si
1,1,98545,20203,otros,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Técnica o tecnológica completa,No
2,2,499179,20212,otros,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,0,Estrato 3,Si,Secundaria (Bachillerato) incompleta,Secundaria (Bachillerato) completa,No
3,3,782980,20195,contaduria publica,SUCRE,Entre 1 millón y menos de 2.5 millones,Entre 21 y 30 horas,Estrato 1,No,Primaria incompleta,Primaria incompleta,No
4,4,785185,20212,administracion de empresas,ATLANTICO,Entre 2.5 millones y menos de 4 millones,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,No


In [16]:
test['ESTU_PRGM_ACADEMICO'].value_counts()

,count
ESTU_PRGM_ACADEMICO,
otros,151235
administracion de empresas,33008
derecho,22984
contaduria publica,22103
psicologia,21256
ingenieria industrial,15228
ingenieria civil,9079
medicina,7915
ingenieria de sistemas,7655


In [17]:
encode = OneHotEncoder()

codificadorPrg = encode.fit_transform(test[['ESTU_PRGM_ACADEMICO']])

arreglo = pd.DataFrame(codificadorPrg.toarray(), columns = encode.categories_)
print(arreglo)
test.drop('ESTU_PRGM_ACADEMICO', axis = 1)

test = pd.concat([test, arreglo], axis = 'columns')
test['ESTU_PRGM_ACADEMICO']

       administracion de empresas contaduria publica derecho ingenieria civil  \
0                             0.0                0.0     0.0              0.0   
1                             0.0                0.0     0.0              0.0   
2                             0.0                0.0     0.0              0.0   
3                             0.0                1.0     0.0              0.0   
4                             1.0                0.0     0.0              0.0   
...                           ...                ...     ...              ...   
296781                        1.0                0.0     0.0              0.0   
296782                        0.0                0.0     1.0              0.0   
296783                        0.0                0.0     1.0              0.0   
296784                        0.0                0.0     0.0              0.0   
296785                        0.0                0.0     0.0              0.0   

       ingenieria de sistem

,ESTU_PRGM_ACADEMICO
0,otros
1,otros
2,otros
3,contaduria publica
4,administracion de empresas
...,...
296781,administracion de empresas
296782,derecho
296783,derecho
296784,otros


In [18]:
test = test.drop('ESTU_PRGM_ACADEMICO', axis = 1)

In [19]:
test.head()

,Unnamed: 0,ID,PERIODO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,...,"(administracion de empresas,)","(contaduria publica,)","(derecho,)","(ingenieria civil,)","(ingenieria de sistemas,)","(ingenieria industrial,)","(licenciatura en pedagogia infantil,)","(medicina,)","(otros,)","(psicologia,)"
0,0,550236,20183,BOLIVAR,Menos de 500 mil,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica completa,Primaria completa,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,98545,20203,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Técnica o tecnológica completa,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,2,499179,20212,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,0,Estrato 3,Si,Secundaria (Bachillerato) incompleta,Secundaria (Bachillerato) completa,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,3,782980,20195,SUCRE,Entre 1 millón y menos de 2.5 millones,Entre 21 y 30 horas,Estrato 1,No,Primaria incompleta,Primaria incompleta,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,785185,20212,ATLANTICO,Entre 2.5 millones y menos de 4 millones,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
test = test.drop('Unnamed: 0', axis = 1)

## Categorización ordinal de las columnas categóricas ordinales

In [25]:
test.columns

Index([                                   'ID',
                                     'PERIODO',
                      'ESTU_PRGM_DEPARTAMENTO',
              'ESTU_VALORMATRICULAUNIVERSIDAD',
                     'ESTU_HORASSEMANATRABAJA',
                        'FAMI_ESTRATOVIVIENDA',
                          'FAMI_TIENEINTERNET',
                         'FAMI_EDUCACIONPADRE',
                         'FAMI_EDUCACIONMADRE',
                    'ESTU_PAGOMATRICULAPROPIO',
               ('administracion de empresas',),
                       ('contaduria publica',),
                                  ('derecho',),
                         ('ingenieria civil',),
                   ('ingenieria de sistemas',),
                    ('ingenieria industrial',),
       ('licenciatura en pedagogia infantil',),
                                 ('medicina',),
                                    ('otros',),
                               ('psicologia',)],
      dtype='object')

## Quitarle los valores nulos a todas las columnas categóricas ordinales


### Conteo de valores nulos

In [26]:
test['FAMI_EDUCACIONMADRE'].value_counts()

,count
FAMI_EDUCACIONMADRE,
Secundaria (Bachillerato) completa,60649
Primaria incompleta,42149
Técnica o tecnológica completa,38571
Educación profesional completa,36477
Secundaria (Bachillerato) incompleta,34566
Primaria completa,24399
Postgrado,20049
Técnica o tecnológica incompleta,11650
Educación profesional incompleta,9647


In [29]:
print(f"Cantidad nulos valor matrícula = {test['ESTU_VALORMATRICULAUNIVERSIDAD'].isna().sum()}")
print(f"Cantidad nulos horas semanales trabaja = {test['ESTU_HORASSEMANATRABAJA'].isna().sum()}")
print(f"Cantidad nulos estrato vivienda = {test['FAMI_ESTRATOVIVIENDA'].isna().sum()}")
print(f"Cantidad nulos educación padre = {test['FAMI_EDUCACIONPADRE'].isna().sum()}")
print(f"Cantidad nulos educación madre = {test['FAMI_EDUCACIONMADRE'].isna().sum()}")

Cantidad nulos valor matrícula = 2723
Cantidad nulos horas semanales trabaja = 13379
Cantidad nulos estrato vivienda = 13795
Cantidad nulos educación padre = 9993
Cantidad nulos educación madre = 10223


### Cambio de valores nulos por 'no info'

In [31]:
test['ESTU_VALORMATRICULAUNIVERSIDAD'] = test['ESTU_VALORMATRICULAUNIVERSIDAD'].fillna("no info")
test['ESTU_HORASSEMANATRABAJA'] = test['ESTU_HORASSEMANATRABAJA'].fillna("no info")
test['FAMI_ESTRATOVIVIENDA'] = test['FAMI_ESTRATOVIVIENDA'].fillna("no info")
test['FAMI_EDUCACIONPADRE'] = test['FAMI_EDUCACIONPADRE'].fillna("no info")
test['FAMI_EDUCACIONMADRE'] = test['FAMI_EDUCACIONMADRE'].fillna("no info")

## Codificación de valores categóricos ordinales.

In [32]:
categories = [
    ['no info', 'No pagó matrícula', 'Menos de 500 mil', 'Entre 500 mil y menos de 1 millón',
     'Entre 1 millón y menos de 2.5 millones', 'Entre 2.5 millones y menos de 4 millones',
     'Entre 4 millones y menos de 5.5 millones', 'Entre 5.5 millones y menos de 7 millones',
     'Más de 7 millones'],  # ESTU_VALORMATRICULAUNIVERSIDAD
    ['no info', '0', 'Menos de 10 horas', 'Entre 11 y 20 horas', 'Entre 21 y 30 horas', 'Más de 30 horas'],  # ESTU_HORASSEMANATRABAJA
    ['no info', 'Sin Estrato', 'Estrato 1', 'Estrato 2', 'Estrato 3', 'Estrato 4', 'Estrato 5', 'Estrato 6'],  # FAMI_ESTRATOVIVIENDA
    ['no info', 'Ninguno', 'No sabe', 'No Aplica', 'Primaria incompleta', 'Primaria completa',
     'Secundaria (Bachillerato) incompleta', 'Secundaria (Bachillerato) completa', 'Técnica o tecnológica incompleta',
     'Técnica o tecnológica completa', 'Educación profesional incompleta', 'Educación profesional completa', 'Postgrado'],  # FAMI_EDUCACIONPADRE
    ['no info', 'Ninguno', 'No sabe', 'No Aplica', 'Primaria incompleta', 'Primaria completa',
     'Secundaria (Bachillerato) incompleta', 'Secundaria (Bachillerato) completa', 'Técnica o tecnológica incompleta',
     'Técnica o tecnológica completa', 'Educación profesional incompleta', 'Educación profesional completa', 'Postgrado'],  # FAMI_EDUCACIONMADRE
]



In [33]:
codificador = OrdinalEncoder(categories=categories)
test[['ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA',
       'FAMI_ESTRATOVIVIENDA', 'FAMI_EDUCACIONPADRE',
       'FAMI_EDUCACIONMADRE']] = codificador.fit_transform(
    test[['ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA',
           'FAMI_ESTRATOVIVIENDA', 'FAMI_EDUCACIONPADRE',
           'FAMI_EDUCACIONMADRE']]
       )

In [34]:
test.head()

,ID,PERIODO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO,"(administracion de empresas,)","(contaduria publica,)","(derecho,)","(ingenieria civil,)","(ingenieria de sistemas,)","(ingenieria industrial,)","(licenciatura en pedagogia infantil,)","(medicina,)","(otros,)","(psicologia,)"
0,550236,20183,BOLIVAR,2.0,2.0,4.0,Si,9.0,5.0,Si,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,98545,20203,ANTIOQUIA,5.0,4.0,3.0,Si,7.0,9.0,No,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,499179,20212,BOGOTÁ,4.0,1.0,4.0,Si,6.0,7.0,No,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,782980,20195,SUCRE,4.0,4.0,2.0,No,4.0,4.0,No,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,785185,20212,ATLANTICO,5.0,3.0,3.0,Si,7.0,7.0,No,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
test['ESTU_PAGOMATRICULAPROPIO'] = test['ESTU_PAGOMATRICULAPROPIO'].fillna('no info')
test['ESTU_PAGOMATRICULAPROPIO'].value_counts()

,count
ESTU_PAGOMATRICULAPROPIO,
No,163869
Si,130110
no info,2807


In [36]:
test = test[test['ESTU_PAGOMATRICULAPROPIO'] != 'no info']

In [37]:
test['ESTU_PAGOMATRICULAPROPIO'].value_counts()

,count
ESTU_PAGOMATRICULAPROPIO,
No,163869
Si,130110


In [38]:
print(f"Cantidad nulos tiene internet = {test['ESTU_PAGOMATRICULAPROPIO'].isna().sum()}")
test['FAMI_TIENEINTERNET'].value_counts()

Cantidad nulos tiene internet = 0


,count
FAMI_TIENEINTERNET,
Si,252138
No,31052


## Conversión One-Hot de las columnas FAMI_TIENEINTERNET y ESTU_PAGOMATRICULAPROPIO

In [39]:
one_hot_encoded = pd.get_dummies(test['ESTU_PAGOMATRICULAPROPIO'], prefix = 'ESTU_PAGOMATRICULAPROPIO')
one_hot_encoded = one_hot_encoded.astype(int)
one_hot_encoded

,ESTU_PAGOMATRICULAPROPIO_No,ESTU_PAGOMATRICULAPROPIO_Si
0,0,1
1,1,0
2,1,0
3,1,0
4,1,0
...,...,...
296781,0,1
296782,1,0
296783,1,0
296784,1,0


In [40]:
test = pd.concat([test, one_hot_encoded], axis = 1)
test = test.drop('ESTU_PAGOMATRICULAPROPIO', axis = 1)

In [41]:
test['ESTU_PAGOMATRICULAPROPIO_No'].head()

,ESTU_PAGOMATRICULAPROPIO_No
0,0
1,1
2,1
3,1
4,1


In [42]:
print(test.columns)

Index([                                   'ID',
                                     'PERIODO',
                      'ESTU_PRGM_DEPARTAMENTO',
              'ESTU_VALORMATRICULAUNIVERSIDAD',
                     'ESTU_HORASSEMANATRABAJA',
                        'FAMI_ESTRATOVIVIENDA',
                          'FAMI_TIENEINTERNET',
                         'FAMI_EDUCACIONPADRE',
                         'FAMI_EDUCACIONMADRE',
               ('administracion de empresas',),
                       ('contaduria publica',),
                                  ('derecho',),
                         ('ingenieria civil',),
                   ('ingenieria de sistemas',),
                    ('ingenieria industrial',),
       ('licenciatura en pedagogia infantil',),
                                 ('medicina',),
                                    ('otros',),
                               ('psicologia',),
                 'ESTU_PAGOMATRICULAPROPIO_No',
                 'ESTU_PAGOMATRICULAPROP

In [43]:
one_hot_encoded = pd.get_dummies(test['FAMI_TIENEINTERNET'], prefix = 'FAMI_TIENEINTERNET')
one_hot_encoded = one_hot_encoded.astype(int)
one_hot_encoded

,FAMI_TIENEINTERNET_No,FAMI_TIENEINTERNET_Si
0,0,1
1,0,1
2,0,1
3,1,0
4,0,1
...,...,...
296781,0,1
296782,0,1
296783,0,1
296784,0,1


In [44]:
test = pd.concat([test, one_hot_encoded], axis = 1)
test = test.drop('FAMI_TIENEINTERNET', axis = 1)

In [45]:
test['FAMI_TIENEINTERNET_Si'].head()

,FAMI_TIENEINTERNET_Si
0,1
1,1
2,1
3,0
4,1


### Eliminamos la columna de departamento para no tenerla en cuenta para este preprocesado

In [46]:
test = test.drop('ESTU_PRGM_DEPARTAMENTO', axis = 1)

In [47]:
test.columns

Index([                                   'ID',
                                     'PERIODO',
              'ESTU_VALORMATRICULAUNIVERSIDAD',
                     'ESTU_HORASSEMANATRABAJA',
                        'FAMI_ESTRATOVIVIENDA',
                         'FAMI_EDUCACIONPADRE',
                         'FAMI_EDUCACIONMADRE',
               ('administracion de empresas',),
                       ('contaduria publica',),
                                  ('derecho',),
                         ('ingenieria civil',),
                   ('ingenieria de sistemas',),
                    ('ingenieria industrial',),
       ('licenciatura en pedagogia infantil',),
                                 ('medicina',),
                                    ('otros',),
                               ('psicologia',),
                 'ESTU_PAGOMATRICULAPROPIO_No',
                 'ESTU_PAGOMATRICULAPROPIO_Si',
                       'FAMI_TIENEINTERNET_No',
                       'FAMI_TIENEINTERN

### Eliminamos la columna periodo y ID ya que se considera, no tiene efecto sobre el rendimiento

In [48]:
test = test.drop('PERIODO', axis = 1)

In [49]:
test.head()

,ID,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,"(administracion de empresas,)","(contaduria publica,)","(derecho,)","(ingenieria civil,)","(ingenieria de sistemas,)","(ingenieria industrial,)","(licenciatura en pedagogia infantil,)","(medicina,)","(otros,)","(psicologia,)",ESTU_PAGOMATRICULAPROPIO_No,ESTU_PAGOMATRICULAPROPIO_Si,FAMI_TIENEINTERNET_No,FAMI_TIENEINTERNET_Si
0,550236,2.0,2.0,4.0,9.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,1,0,1
1,98545,5.0,4.0,3.0,7.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,0,0,1
2,499179,4.0,1.0,4.0,6.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,0,0,1
3,782980,4.0,4.0,2.0,4.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,1,0
4,785185,5.0,3.0,3.0,7.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,1


In [51]:
test.to_csv('test1.csv', index = False)

In [54]:
!head test1.csv

ID,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,"('administracion de empresas',)","('contaduria publica',)","('derecho',)","('ingenieria civil',)","('ingenieria de sistemas',)","('ingenieria industrial',)","('licenciatura en pedagogia infantil',)","('medicina',)","('otros',)","('psicologia',)",ESTU_PAGOMATRICULAPROPIO_No,ESTU_PAGOMATRICULAPROPIO_Si,FAMI_TIENEINTERNET_No,FAMI_TIENEINTERNET_Si
550236,2.0,2.0,4.0,9.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,1,0,1
98545,5.0,4.0,3.0,7.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,0,0,1
499179,4.0,1.0,4.0,6.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,0,0,1
782980,4.0,4.0,2.0,4.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,1,0
785185,5.0,3.0,3.0,7.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,1
58495,3.0,5.0,2.0,1.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,1,0,1
705444,2.0,1.0,4.0,5.0,9.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,1
557548,8.0,